# 6.1 텍스트 데이터 다루기
이 장에서는 텍스트, 시계열 또는 일반적인 시퀀스 데이터를 처리할 수 있는 딥러닝 모델을 살펴봅니다.
일반적으로 시퀀스 데이터를 처리하는 딥러닝 모델은 두가지입니다.
1. 순환 신경망
2. 1D Convnet

이런 알고리즘은 아래와 같은 경우에 적용가능합니다.
1. 문서분류나 시계열 분류
2. 시계열 비교
3. Seq-to-Seq 학습
4. 감성 분석
5. 시계열 예측

## 텍스트 데이터 다루기
문자 언어에 대한 통계적 구조를 만들어 텍스트를 분석합니다. 다시 말해 컴퓨터 비전이 픽셀을 이용한 패턴 인식이라면, 자연어 처리를 위한 딥러닝 모델은 단어, 문장, 문단에 적용한 패턴인식입니다. 이러한 패턴을 기계가 학습하기 위해 먼저 텍스트 벡터화를 진행합니다. 벡터화에는 대표적으로 세가지 방법이 있습니다.

### 텍스트 벡터화
1. 텍스트를 단어로 나누고, 각 단어를 하나의 벡터화
2. 텍스트를 문자로 나누고, 각 문자를 하나의 벡터화
3. 단어나 문자의 n-gram을 추출하여, 각 n-gram를 하나의 벡터화

이렇게 변환하는 작업을 "토큰화" 라고 하며, 변환된 텍스트를 "토큰" 이라고 합니다.
이 토큰을 벡터와 연결짓는 작업은 대표적으로 두가지 방법이 있습니다.

1. One-hot encoding
2. token(word) Embedding

## 1. One-hot encoding

In [5]:
#one-hot encoding easy example
import numpy as np

samples = ['The cat sat in the mat.', 'The dog ate my homework.']

token_index = {}
for sample in samples:
    for word in sample.split():
        if word not in token_index:
            token_index[word] = len(token_index) + 1

max_length = 10

results = np.zeros(shape=(len(samples), max_length, max(token_index.values()) + 1 ))

for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = token_index.get(word)
        results[i, j, index] = 1.

results

array([[[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0

In [19]:
import string

samples = ['The cat sat in the mat.', 'The dog ate my homework.']
characters = string.printable
token_index = dict(zip(characters, range(1, len(characters) + 1)))

max_length = len(characters)
results = np.zeros((len(samples), max_length, max(token_index.values()) + 1))
for i, sample in enumerate(samples):
    for j, character in enumerate(characters):
        index = token_index.get(character)
        #print(index)
        results[i, j, index] = 1.
print(results.shape)

(2, 100, 101)


In [20]:
#one-hot encoding using keras
from keras.preprocessing.text import Tokenizer

samples = ['The cat sat in the mat.', 'The dog ate my homework.']

tokenizer = Tokenizer(num_words = 1000)
tokenizer.fit_on_texts(samples)

sequences = tokenizer.texts_to_sequences(samples)

one_hot_results = tokenizer.texts_to_matrix(samples, mode = 'binary')

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

9개의 고유한 토큰을 찾았습니다.


In [22]:
#one-hot hashing
samples = ['The cat sat in the mat.', 'The dog ate my homework.']

dimensionality = 1000
max_length = 10

results = np.zeros((len(samples), max_length, dimensionality))
for i, sample in enumerate(samples):
    for j, word in list(enumerate(sample.split()))[:max_length]:
        index = abs(hash(word)) %dimensionality
        results[i, j, index] = 1.
print(results.shape)

(2, 10, 1000)


## 2. Word Embedding

One-hot encoding이나 Hashing으로 얻은 단어 표현은 Sparse 하다는 문제점이 있는 반면, Word Embedding의 경우 조밀하고 비교적 저차원이다.

단어 임베딩을 만드는 방법으로 두가지 입니다
1. 관심 대상인 문제와 함께 단어 임베딩을 학습
2. 다른 머신 러닝 작업에서 미리 계산된 단어 임베딩을 로드(pretrained word embedding)

유사한 단어가 유사한 공간에 배치되도록 학습하는 것이 중요한 포인트이다.

In [23]:
from keras.layers import Embedding

embedding_layer = Embedding(1000, 64)

In [26]:
from keras.datasets import imdb
from keras import preprocessing
import numpy as np

# modify the default parameters of np.load
np_load_old = np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)


max_features = 10000
maxlen = 20

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

x_train = preprocessing.sequence.pad_sequences(x_train, maxlen = maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen = maxlen)

In [27]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()
model.add(Embedding(10000, 8, input_length = maxlen))

model.add(Flatten())

model.add(Dense(1, activation = 'sigmoid'))
model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics=['acc'])
model.summary()

history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________
Instructions for updating:
Use tf.cast instead.
Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 66us/step - loss: 0.6759 - acc: 0.6050 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 1s 37us/step - loss: 0.5657 - acc: 0.7

In [31]:
import os

imdb_dir = 'C:/Users/Gene/Desktop/projects/keras_tutorial/datasets/aclImdb'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        f = open(os.path.join(dir_name, fname), encoding = "utf8")
        texts.append(f.read())
        f.close()
        
        if label_type == 'neg':
            labels.append(0)
        else:
            labels.append(1)